In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy.spatial.distance import cdist
from skimage import measure
from matplotlib import pyplot as plt
import os
import tifffile
from tqdm import tqdm
from typing import List, Tuple

In [2]:
selected_magnification = "20x"
selected_image_type = "raw"
raw_thr = .09
max_dist = 9
aspect = np.array((1/3, 1/3))    # XY: 20x / aspect => 60x

### Read shift data

In [3]:
shifts = pd.read_csv(f"shift_correction/{selected_magnification}_{selected_image_type}.shifts.csv")
shifts.index = shifts["sid"].values
shifts.drop("sid", 1, inplace=True)

# Matching 20x_dw and reference dots

In [4]:
dots_data = pd.read_csv("/mnt/data/Imaging/202105-Deconwolf/data_210726/dots_data.clean.tsv.gz", sep="\t")
dots_data = dots_data[selected_magnification == dots_data["magnification"]]
dots_data = dots_data[selected_image_type == dots_data["image_type"]]

In [5]:
reference = pd.read_csv("../../data/60x_reference/ref__dw.global_thr.tsv", sep="\t")

In [6]:
def get_unassigned_count(pdist: np.ndarray) -> int:
    return np.sum(np.nansum(np.isnan(pdist), 1) != pdist.shape[1])

def match_dots(dotsA: np.ndarray, dotsB: np.ndarray) -> List[Tuple[int, int, float]]:
    pdist = cdist(dotsA, dotsB)
    
    matched: List[Tuple[int, int, float]] = []
    unassigned_rows = pdist.shape[0] + 1
    iteration_counter = 0
    print("Iteration #", end="")
    while unassigned_rows > get_unassigned_count(pdist) and 0 < get_unassigned_count(pdist):
        iteration_counter += 1
        print(f"...{iteration_counter}", end="")

        unassigned_rows = get_unassigned_count(pdist)
        A_id = 0
        while A_id < pdist.shape[0]:
            if pdist.shape[1] == np.sum(np.isnan(pdist[A_id, :])):
                A_id += 1
                continue
            B_id = np.nanargmin(pdist[A_id, :])
            if np.nanargmin(pdist[:, B_id]) == A_id:
                matched.append((A_id, B_id, pdist[A_id, B_id]))
                pdist[A_id, :] = np.nan
                pdist[:, B_id] = np.nan
            A_id += 1
    print("")
    return matched

In [7]:
match_output: List[pd.DataFrame] = []
match_output_full: List[pd.DataFrame] = []
match_counts: List[Tuple[int, int, int, int]] = []
for current_field_id in tqdm(np.unique(dots_data["sid"])):    
    dots = dots_data.loc[current_field_id == dots_data["sid"], :].copy(
        ).sort_values("Value2", ascending=False).reset_index(drop=True)
    
    dot_coords = dots.loc[raw_thr <= dots["Value2"], ("x", "y")].copy().reset_index(drop=True)
    dot_coords2 = dot_coords.copy() / aspect
    dot_coords2["x"] += (shifts.loc[current_field_id, "x"] * 9)
    dot_coords2["y"] += (shifts.loc[current_field_id, "y"] * 9)
    ref_coords = reference.loc[reference["sid"] == current_field_id, ("x", "y")].copy().reset_index(drop=True)
    
    matched: List[Tuple[int, int, float]] = match_dots(dot_coords2, ref_coords)
    
    matched_a = np.array(matched)
    matched_field = pd.DataFrame(dict(
        series=current_field_id,
        id_20x=matched_a[:, 0].astype("i"),
        id_60x=matched_a[:, 1].astype("i"),
        eudist=matched_a[:, 2]
    ))
    matched_field_within_dist = matched_field[matched_field["eudist"] <= max_dist]
    match_output.append(matched_field_within_dist)
    match_output_full.append(matched_field)
    match_counts.append((current_field_id,
                         matched_field_within_dist.shape[0],
                         matched_field.shape[0]-matched_field_within_dist.shape[0],
                         dot_coords2.shape[0],
                         ref_coords.shape[0]))

  0%|          | 0/7 [00:00<?, ?it/s]

Iteration #...1...2...3...4...5...6...7

 14%|█▍        | 1/7 [00:01<00:07,  1.28s/it]


Iteration #...1...2...3...4...5...6...7...8

 29%|██▊       | 2/7 [00:03<00:08,  1.68s/it]

...


Iteration #...1...2...3...4...5...6...7...8...9...10...11...12...13

 86%|████████▌ | 6/7 [00:11<00:01,  1.87s/it]

...14...15...16
Iteration #...1...2...3...4...5...6...7...8...9...10...11...12...13...14...15

100%|██████████| 7/7 [00:12<00:00,  1.72s/it]

...16...17...18...19...20...21...22...23


In [8]:
match_counts_a = np.array(match_counts)
n_total_matched = np.sum(match_counts_a[:, 1])
n_total_over_dist = np.sum(match_counts_a[:, 2])
n_total_20x = np.sum(match_counts_a[:, 3])
n_total_60x = np.sum(match_counts_a[:, 4])
print(f"{n_total_matched}/{n_total_20x} ({n_total_matched/n_total_20x*100:.2f}%) 20x dots matched to 60x")
print(f"{n_total_matched}/{n_total_60x} ({n_total_matched/n_total_60x*100:.2f}%) 60x dots matched to 20x")
print(f"{n_total_over_dist} ({n_total_over_dist/n_total_20x*100:.2f}% 20x, {n_total_over_dist/n_total_60x*100:.2f}% 60x) dots matched with distance > {max_dist}")

6447/15685 (41.10%) 20x dots matched to 60x
6447/9439 (68.30%) 60x dots matched to 20x
2992 (19.08% 20x, 31.70% 60x) dots matched with distance > 9


In [9]:
for series_id in range(match_counts_a.shape[0]):
    n_matched = match_counts_a[series_id, 1]
    n_over_dist = match_counts_a[series_id, 2]
    n_20x = match_counts_a[series_id, 3]
    n_60x = match_counts_a[series_id, 4]
    print(f"""Field #{series_id+1}
    {n_matched}/{n_20x} ({n_matched/n_20x*100:.2f}%) 20x dots matched to 60x
    {n_matched}/{n_60x} ({n_matched/n_60x*100:.2f}%) 60x dots matched to 20x
    {n_over_dist} ({n_over_dist/n_20x*100:.2f}% 20x, {n_over_dist/n_60x*100:.2f}% 60x) dots matched with distance > {max_dist}
    """)

Field #1
    1083/2370 (45.70%) 20x dots matched to 60x
    1083/1308 (82.80%) 60x dots matched to 20x
    225 (9.49% 20x, 17.20% 60x) dots matched with distance > 9
    
Field #2
    1539/3319 (46.37%) 20x dots matched to 60x
    1539/1921 (80.11%) 60x dots matched to 20x
    382 (11.51% 20x, 19.89% 60x) dots matched with distance > 9
    
Field #3
    715/1623 (44.05%) 20x dots matched to 60x
    715/1045 (68.42%) 60x dots matched to 20x
    330 (20.33% 20x, 31.58% 60x) dots matched with distance > 9
    
Field #4
    1289/2727 (47.27%) 20x dots matched to 60x
    1289/1548 (83.27%) 60x dots matched to 20x
    259 (9.50% 20x, 16.73% 60x) dots matched with distance > 9
    
Field #5
    1226/3956 (30.99%) 20x dots matched to 60x
    1226/2404 (51.00%) 60x dots matched to 20x
    1178 (29.78% 20x, 49.00% 60x) dots matched with distance > 9
    
Field #6
    477/1101 (43.32%) 20x dots matched to 60x
    477/691 (69.03%) 60x dots matched to 20x
    214 (19.44% 20x, 30.97% 60x) dots match

#### General statistics, excluding field #7

In [10]:
match_counts_a = np.array(match_counts)
n_total_matched = np.sum(match_counts_a[:6, 1])
n_total_over_dist = np.sum(match_counts_a[:6, 2])
n_total_20x = np.sum(match_counts_a[:6, 3])
n_total_60x = np.sum(match_counts_a[:6, 4])
print(f"{n_total_matched}/{n_total_20x} ({n_total_matched/n_total_20x*100:.2f}%) 20x dots matched to 60x")
print(f"{n_total_matched}/{n_total_60x} ({n_total_matched/n_total_60x*100:.2f}%) 60x dots matched to 20x")
print(f"{n_total_over_dist} ({n_total_over_dist/n_total_20x*100:.2f}% 20x, {n_total_over_dist/n_total_60x*100:.2f}% 60x) dots matched with distance > {max_dist}")

6329/15096 (41.93%) 20x dots matched to 60x
6329/8917 (70.98%) 60x dots matched to 20x
2588 (17.14% 20x, 29.02% 60x) dots matched with distance > 9


#### General statistics, excluding fields #6-7

In [11]:
match_counts_a = np.array(match_counts)
n_total_matched = np.sum(match_counts_a[:5, 1])
n_total_over_dist = np.sum(match_counts_a[:5, 2])
n_total_20x = np.sum(match_counts_a[:5, 3])
n_total_60x = np.sum(match_counts_a[:5, 4])
print(f"{n_total_matched}/{n_total_20x} ({n_total_matched/n_total_20x*100:.2f}%) 20x dots matched to 60x")
print(f"{n_total_matched}/{n_total_60x} ({n_total_matched/n_total_60x*100:.2f}%) 60x dots matched to 20x")
print(f"{n_total_over_dist} ({n_total_over_dist/n_total_20x*100:.2f}% 20x, {n_total_over_dist/n_total_60x*100:.2f}% 60x) dots matched with distance > {max_dist}")

5852/13995 (41.81%) 20x dots matched to 60x
5852/8226 (71.14%) 60x dots matched to 20x
2374 (16.96% 20x, 28.86% 60x) dots matched with distance > 9


### Export

In [12]:
matched_dots = pd.concat(match_output)

In [13]:
matched_dots.to_csv(os.path.join("../../data/magnifications_matching",
                                 f"{selected_magnification}_{selected_image_type}.matched_dots.global_thr.tsv"
                                ), sep="\t", index=False)

In [14]:
pd.DataFrame(match_counts_a, columns=["series_id", "matched", "matched_over_dist", "raw_total", "dw_total"]
            ).to_csv(os.path.join("../../data/magnifications_matching",
                                  f"{selected_magnification}_{selected_image_type}.matched_dots.global_thr.counts.tsv"
                                 ), sep="\t", index=False)